# 2.00 Object Relational Mapping
This notebook demonstrates how we can use object relational mapping with Postgres.

It uses sqlalchemy as ORM. Install via `pip3 install sqlalchemy` or better, to install alongside the flask plugin use
`pip3 install flask_sqlalchemy`

In [ ]:
import sqlalchemy
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import Column, Integer, String

In [ ]:
# use optional echo=True to output commands
db = sqlalchemy.create_engine('postgresql://localhost/cs6', echo=True)

In [ ]:
Base = declarative_base()

In [ ]:
class Animal(Base):
    __tablename__ = 'animals'
    
    id = Column(Integer, primary_key=True)
    name = Column(String)
    type = Column(String)
    
    def __repr__(self):
        return '{}({})'.format(self.name, self.type)

In [ ]:
Animal(name='Tux', type='penguin')

In [ ]:
Base.metadata.create_all(db)

## 2.01 Communicating with the database
To communicate with the database, we need to create a session object

In [ ]:
from sqlalchemy.orm import sessionmaker

In [ ]:
Session = sessionmaker(bind=db)

In [ ]:
session = Session()

Let's say we want to add two new penguins to our database. We can use the above defined class to create instances.

In [ ]:
tux = Animal(name='Tux', type='penguin')
tango = Animal(name='Tango', type='penguin')

In the next step, we add these objects to the session. So far no SQL queries have been issued to the database.

In [ ]:
session.add(tux)
session.add(tango)

However, when calling `commit` all changes in the session are flushed to the database.

In [ ]:
session.commit()

To retrieve data from the database, we can use sqlalchemy as well:

In [ ]:
for animal in session.query(Animal).order_by(Animal.name):
    print(animal.name, animal.type)

For a complete tutorial on how to use SQLAlchemy accessors see https://docs.sqlalchemy.org/en/13/orm/tutorial.html

## 2.02 Relationships

Imagine we want to write a simple to do list for users. How could we do that using a database?


==> This is a 1 User <=> N todos relationship

More on relationships: <https://docs.sqlalchemy.org/en/13/orm/relationships.html>

In [ ]:
from sqlalchemy import ForeignKey
from sqlalchemy.orm import relationship

In [ ]:
class User(Base):
    __tablename__ = 'users'
    
    id = Column(Integer, primary_key=True)
    name = Column(String)
    
    todos = relationship('Todo', back_populates='user')

In [ ]:
class Todo(Base):
    __tablename__ = 'todos'
    
    id = Column(Integer, primary_key=True)
    note = Column(String)
    
    user_id = Column(Integer, ForeignKey('users.id'))
    user = relationship('User', back_populates='todos')

Create necessary tables

In [ ]:
Base.metadata.create_all(db)

Now we can start retrieving a user

In [ ]:
tux = User(name='Tux')

In [ ]:
tux.todos.append(Todo(note='Go fishing'))
tux.todos.append(Todo(note='Go see sealion'))

In [ ]:
session.add(tux)

In [ ]:
session.commit()

Again, we can make use of SQLAlchemy's builtin mechanisms to retrieve data as objects!

In [ ]:
for todo in session.query(Todo):
    print('{} has "{}" to do!'.format(todo.user.name, todo.note))